<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/transformers_demo/text_generation_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install langchain-community langchain-google-genai langchain google-generativeai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.runnables import RunnablePassthrough

# 1. Prompt
template = """<s><|user|>
current conversation:
{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["chat_history", "input_prompt"]
)

# 2. Memory
memory = ConversationBufferWindowMemory(
    k=2,
    memory_key="chat_history",
    return_messages=False
)

# 3. Chain
chain = (
    RunnablePassthrough.assign(
        chat_history=lambda _: memory.load_memory_variables({})["chat_history"]
    )
    | prompt
    | llm
)

In [ ]:
user_input = "hello, my name is maarten. what is 1 + 1?"
result = chain.invoke({"input_prompt": user_input})
memory.save_context({"input_prompt": user_input}, {"output": result.content})
print(result.content) # That would be 2.

In [ ]:
user_input = "what is 3 + 1?"
result = chain.invoke({"input_prompt": user_input})
memory.save_context({"input_prompt": user_input}, {"output": result.content})
print(result.content) # That would be 4.

In [ ]:
user_input = "what is my name?"
result = chain.invoke({"input_prompt": user_input})
memory.save_context({"input_prompt": user_input}, {"output": result.content})
print(result.content) # Your name is Maarten.

In [ ]:
user_input = "what is my name?"
result = chain.invoke({"input_prompt": user_input})
print(result.content) # I don't know your name. You haven't told me yet!

In [ ]:
print("Internal memory:", memory.chat_memory.messages)
print("Memory injected into prompt:", memory.load_memory_variables({}))